In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/lending-club-dataset/lending_club_loan_two.csv")
df = df.dropna()

In [ ]:
df["loan_status"]

In [ ]:
sns.countplot(x="loan_status", data=df)

In [ ]:
df["Fully Paid"] = pd.get_dummies(df["loan_status"], drop_first=True)
df = df.drop("loan_status", axis=1)

In [ ]:
#pd.get_dummies(df["home_ownership"], drop_first=True)
#df["home_ownership"].value_counts()
df = pd.concat([df,pd.get_dummies(df["home_ownership"], drop_first=True)], axis=1)
df = df.drop("home_ownership", axis=1)

df = pd.concat([df,pd.get_dummies(df["sub_grade"], drop_first=True)], axis=1)
df = df.drop("sub_grade", axis=1)

df = pd.concat([df,pd.get_dummies(df["grade"], drop_first=True)], axis=1)
df = df.drop("grade", axis=1)

#df = pd.concat([df,pd.get_dummies(df["purpose"], drop_first=True)], axis=1)
df = df.drop("purpose", axis=1)

#df = pd.concat([df,pd.get_dummies(df["emp_length"], drop_first=True)], axis=1)
df = df.drop("emp_length", axis=1)

df = pd.concat([df,pd.get_dummies(df["term"], drop_first=True)], axis=1)
df = df.drop("term", axis=1)

df = pd.concat([df,pd.get_dummies(df["verification_status"], drop_first=True)], axis=1)
df = df.drop("verification_status", axis=1)

#df = pd.concat([df,pd.get_dummies(df["application_type"], drop_first=True)], axis=1)
df = df.drop("application_type", axis=1)

In [ ]:
df.corr()["Fully Paid"][:-1].sort_values().plot(kind="bar", figsize=(20, 10))

In [ ]:
df = df.drop("address", axis=1)
df = df.drop("issue_d", axis=1)
df = df.drop("emp_title", axis=1)
df = df.drop("title", axis=1)
df = df.drop("earliest_cr_line", axis=1)
df = df.drop("initial_list_status", axis=1)

In [ ]:
# settings to display all columns
pd.set_option("display.max_columns", None)
# display the dataframe head
df.head()

<h1>Scaling and splitting</h1>

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.drop('Fully Paid',axis=1).values
y = df['Fully Paid'].values

In [ ]:
print(df["annual_inc"].max())
print(df["annual_inc"].min())


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X_train)
#

In [ ]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train[0]

## Creating the Model



In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation,Dropout

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

In [ ]:
X_train.shape

In [ ]:
from tensorflow.keras import optimizers

#optimizer = optimizers.Adam()
optimizer = optimizers.Adam(clipvalue=.5)

In [ ]:
model = Sequential()

# https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw

model.add(Dense(units=60,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.2))


#model.add(Dense(units=42,activation='relu'))
#model.add(Dropout(0.5))

model.add(Dense(units=1,activation='sigmoid'))

# For a binary classification problem
model.compile(loss='binary_crossentropy', optimizer=optimizer)

In [ ]:
model.fit(x=X_train, 
          y=y_train, 
          epochs=600,
          validation_data=(X_test, y_test), verbose=1,
          callbacks=[early_stop]
          )

In [ ]:
pd.DataFrame(model.history.history).plot()